In [1]:
!pip install opencv-python pickle5 dlib face-recognition boto3 pymongo

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pickle5
Failed to build pickle5


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [54 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\pickle5
      copying pickle5\pickle.py -> build\lib.win-amd64-cpython-312\pickle5
      copying pickle5\pickletools.py -> build\lib.win-amd64-cpython-312\pickle5
      copying pickle5\__init__.py -> build\lib.win-amd64-cpython-312\pickle5
      creating build\lib.win-amd64-cpython-312\pickle5\test
      copying pickle5\test\pickletester.py -> build\lib.win-amd64-cpython-312\pickle5\test
      copying pickle5\test\test_pickle.py -> build\lib.win-amd64-cpython-312\pickle5\test
      copying pickle5\test\test_picklebuffer.py -> build\lib.win-amd64-cpython-312\pickle5\test
      copying pickle5\test\__init__.py -> build\lib.win-amd64-cpython-312\pickle5\test
      running build_ext
      building 'pickle5._pickle' extension
      crea

In [30]:
import boto3
from pymongo import MongoClient
import pickle

In [ ]:
mongo_client = MongoClient("mongodb://localhost:27017/", serverSelectionTimeoutMS=500000  )
db = mongo_client["test"]


In [32]:
from pymongo import MongoClient
from bson import ObjectId
from pymongo.errors import ConnectionFailure

def connect_mongodb():
    """Establish connection to MongoDB"""
    try:
        client = MongoClient(
            "mongodb://localhost:27017/",
            serverSelectionTimeoutMS=5000
        )
        client.server_info()  # Test connection
        return client
    except ConnectionFailure as e:
        print(f"❌ MongoDB Connection Failed: {e}")
        return None

def get_folder_by_id(collection, folder_id):
    """Fetch a single document by _id"""
    
    return collection.find_one({"_id": ObjectId(folder_id)})
   

def get_all_folders(collection):
    """Fetch all documents in collection"""
    return list(collection.find())

def main():
    # Connect to MongoDB
    client = connect_mongodb()
    if not client:
        return

    try:
        # Access database and collection
        db = client["test"]  # Database name
        collection = db["folders"]  # Collection name

        # Example 1: Get document by ID
        doc_id = "67f4f5a0f15d10c969a4eacd"  # Replace with actual ID
        folder = get_folder_by_id(collection, doc_id)
        if folder:
            print(f"📂 Found folder:\n{folder}")

        # Example 2: Get all documents
        all_folders = get_all_folders(collection)
        print(f"\n📚 Total folders found: {len(all_folders)}")
        
    finally:
        client.close()
        print("\n🔌 MongoDB connection closed")

if __name__ == "__main__":
    main()

📂 Found folder:
{'_id': ObjectId('67f4f5a0f15d10c969a4eacd'), 'name': 'Vizag Trip', 'client': 'Arman Pani', 'purpose': 'Winter Vacation Trip', 'photoCount': 0, 'webLink': 'wedlinker.com/c30ddc49-a1a5-47f1-86c3-520dee00277e', 'photos': ['https://event-images-by-photographers.s3.us-east-1.amazonaws.com/uploads/f9a48ae6-a010-4f3e-b3df-d130d6e6c903-1.jpg', 'https://event-images-by-photographers.s3.us-east-1.amazonaws.com/uploads/2dc2bba0-6a37-4730-bdd8-4a3fac69b023-2.jpg', 'https://event-images-by-photographers.s3.us-east-1.amazonaws.com/uploads/1be627d0-a68b-4c53-87bc-f12d4cf98782-5.jpg', 'https://event-images-by-photographers.s3.us-east-1.amazonaws.com/uploads/1150e341-eb78-44a9-8a9a-39e830ad8f00-6.jpg', 'https://event-images-by-photographers.s3.us-east-1.amazonaws.com/uploads/c0d3520d-a73e-42e3-b9d3-987b95b93371-8.jpg', 'https://event-images-by-photographers.s3.us-east-1.amazonaws.com/uploads/c62cdb2a-6178-4661-972b-7057f60e4931-9.jpg', 'https://event-images-by-photographers.s3.us-east-

In [39]:
from bson import ObjectId
import cv2
import numpy as np
import dlib
from botocore.exceptions import ClientError
from face_recognition import face_encodings, face_locations
def createEncodings(image):
    """
    Generate face encodings for the given image.
    """
    # Detect faces in the image
    detected_faces = face_locations(image)
    
    #Create encodings for all faces in an image
    known_encodings=face_encodings(image,known_face_locations=detected_faces)
    return known_encodings, detected_faces
def process_folder_by_id(folder_id):
    collection = db["folders"]
    
    # Fetch the folder document
    folder_doc = collection.find_one({"_id": ObjectId(folder_id)})
    if not folder_doc or "photos" not in folder_doc:
        print("Folder not found or has no photos")
        return
    
    updated_photos = []
    for photo in folder_doc["photos"]:
        if isinstance(photo, str):  # Handle legacy string format
            photo = {"url": photo, "encodings": []}
        
        try:
            # 1. Download image from S3
            key = photo["url"].split("uploads/")[-1]
            response = s3.get_object(
                Bucket='event-images-by-photographers',
                Key=f"uploads/{key}"
            )
            image_data = response['Body'].read()
            
            # 2. Decode image
            image = cv2.imdecode(np.frombuffer(image_data, np.uint8), cv2.IMREAD_COLOR)
            if image is None:
                raise ValueError("Invalid image data")
            
            # 3. Generate encodings for THIS PHOTO
            encodings, _ = createEncodings(image)
            encodings_bin = [pickle.dumps(enc) for enc in encodings]
            
            # 4. Update photo entry
            updated_photo = {
                "url": photo["url"],
                "encodings": encodings_bin  # Store encodings per photo
            }
            
        except Exception as e:
            print(f"Error processing {photo['url']}: {e}")
            updated_photo = photo  # Preserve original data
        
        updated_photos.append(updated_photo)
    
    # 5. Update ALL PHOTOS in the folder document
    collection.update_one(
        {"_id": ObjectId(folder_id)},
        {"$set": {"photos": updated_photos}}  # Overwrite the entire photos array
    )
    print(f"Updated encodings for {len(updated_photos)} photos")

In [40]:
from face_recognition import compare_faces, face_distance
from bson import ObjectId

def compareFaceEncodings(unknown_encoding,known_encodings,known_names):
    """
    Compares face encodings to check if 2 faces are same or not.

    Parameters
    ----------
    unknown_encoding : np array
        Face encoding of unknown people.
    known_encodings : np array
        Face encodings of known people.
    known_names : list of strings
        Names of known people

    Returns
    -------
    acceptBool : Bool
        face matched or not
    duplicateName : String
        Name of matched face
    distance : Float
        Distance between 2 faces

    """
    duplicateName=""
    distance=0.0
    matches = compare_faces(known_encodings, unknown_encoding,tolerance=0.5)
    face_distances = face_distance(known_encodings, unknown_encoding)
    best_match_index = np.argmin(face_distances)
    distance=face_distances[best_match_index]
    if matches[best_match_index]:
        acceptBool=True
        duplicateName=known_names[best_match_index]
    else:
        acceptBool=False
        duplicateName=""
    return acceptBool,duplicateName,distance


def query_selfie(selfie_image_path, target_folder_id):
    """
    Return URLs of photos where user's face appears.
    """
    # 1. Load selfie
    selfie_image = cv2.imread(selfie_image_path)
    if selfie_image is None:
        return []
    
    # 2. Get selfie encoding
    selfie_encodings, _ = createEncodings(selfie_image)
    if not selfie_encodings:
        return []
    selfie_encoding = selfie_encodings[0]

    # 3. Get folder data
    collection = db["folders"]
    folder_doc = collection.find_one({"_id": ObjectId(target_folder_id)})
    if not folder_doc:
        return []

    # 4. Check each photo
    matching_urls = []
    for photo in folder_doc.get("photos", []):
        if "encodings" not in photo:
            continue
            
        # Compare against all encodings in this photo
        for enc_bin in photo["encodings"]:
            enc = pickle.loads(enc_bin)
            match = compare_faces([enc], selfie_encoding, tolerance=0.5)[0]
            if match:
                matching_urls.append(photo["url"])
                break  # No need to check other encodings in this photo
    
    return matching_urls

In [41]:
if __name__ == "__main__":
    # Step 1: Preprocess a folder (run once when images are added)
    process_folder_by_id("67f5c24f05990e4a88eacca8")
    
    # Step 2: User uploads selfie and queries
    result_urls = query_selfie("test_selfie.jpg", "67f5c24f05990e4a88eacca8")
    print("Matching images:", result_urls)

Updated encodings for 10 photos
Matching images: []
